In [141]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pprint import pprint
from lxml import html
from lxml.html import fromstring
import urllib.request
from urllib.request import urlopen
import random
import re
import time
#import scrapy

In [137]:
#DATAFRAME RESET 
df = df.iloc[0:0]

In [138]:
weeks=3
url='https://www.kino-zeit.de/filme/filmstarts/aktuell'
df=pd.DataFrame(columns=['start','film','director','actor','link','image','distro','country','trailer'])

In [ ]:
####HOLD DONT TOUCH###
def make_soup(url):
    html=requests.get(url).content
    return BeautifulSoup(html,'html.parser')

def next_url(soup):
    url=''.join([f'https://www.kino-zeit.de/filme/filmstarts/{i["href"]}' for i in soup.find_all("a",{"class": "next btn btn-primary"})])
    return url

def how_much_soup(url):
    for i in range(1,weeks): #Input fixed how many weeks we are going to check
        print(url)
        soup= make_soup(url)
        df_loop=pd.DataFrame(columns=['start','film','director','actor','link','image','distro','country','trailer'])
        df_loop['film']=[i.text for i in soup.find_all(['h3','a'],{'class': 'filmlink'})]
        df_loop['film']=df_loop['film'].str.replace('\(\d\d\d\d\)','')
        df_loop['start']=[i.text for i in soup.find_all('div',{'class':'col-md-2 infospalte-1'})]
        df_loop['start']= re.findall(r'\d\d.\d\d.\d\d\d\d', str(df_loop['start']))
        df_loop['link']=[f'https://www.kino-zeit.de{i["href"]}' for i in soup.find_all(["h3","a"],{"class": "filmlink"})]
        url=next_url(soup)#soll Link vorbereiten bevor loop nächste Runde für nächste Woche dreht
        print(url)
        return (df_loop)

df=df.append(how_much_soup(url))


In [139]:
####HOLD DONT TOUCH###
def make_soup(url):
    html=requests.get(url).content
    return BeautifulSoup(html,'html.parser')

def next_url(soup):
    url=''.join([f'https://www.kino-zeit.de/filme/filmstarts/{i["href"]}' for i in soup.find_all("a",{"class": "next btn btn-primary"})])
    return url

def how_much_soup(url):
    for i in range(1,weeks): #Input fixed how many weeks we are going to check
        print(url)
        soup= make_soup(url)
        df_loop=pd.DataFrame(columns=['start','film','director','actor','link','image','distro','country','trailer'])
        df_loop['film']=[i.text for i in soup.find_all(['h3','a'],{'class': 'filmlink'})]
        df_loop['film']=df_loop['film'].str.replace('\(\d\d\d\d\)','')
        df_loop['start']=[i.text for i in soup.find_all('div',{'class':'col-md-2 infospalte-1'})]
        df_loop['start']= re.findall(r'\d\d.\d\d.\d\d\d\d', str(df_loop['start']))
        df_loop['link']=[f'https://www.kino-zeit.de{i["href"]}' for i in soup.find_all(["h3","a"],{"class": "filmlink"})]
        url=next_url(soup)#soll Link vorbereiten bevor loop nächste Runde für nächste Woche dreht
        print(url)
        return (df_loop)

df=df.append(how_much_soup(url))


https://www.kino-zeit.de/filme/filmstarts/aktuell
https://www.kino-zeit.de/filme/filmstarts/2020-10-08


In [140]:
df

,start,film,director,actor,link,image,distro,country,trailer
0,01.10.2020,Contemporary Past - Die Gegenwart der Vergange...,NaN,NaN,https://www.kino-zeit.de/node/50404,NaN,NaN,NaN,NaN
1,01.10.2020,Die Misswahl - Der Beginn einer Revolution,NaN,NaN,https://www.kino-zeit.de/node/48346,NaN,NaN,NaN,NaN
2,01.10.2020,Drachenreiter,NaN,NaN,https://www.kino-zeit.de/node/49005,NaN,NaN,NaN,NaN
3,01.10.2020,Enfant Terrible,NaN,NaN,https://www.kino-zeit.de/node/48025,NaN,NaN,NaN,NaN
4,01.10.2020,Good Kisser,NaN,NaN,https://www.kino-zeit.de/node/50833,NaN,NaN,NaN,NaN
5,01.10.2020,"Gott, du kannst ein Arsch sein!",NaN,NaN,https://www.kino-zeit.de/node/50476,NaN,NaN,NaN,NaN
6,01.10.2020,Hurtigruten und Norwegen,NaN,NaN,https://www.kino-zeit.de/node/50852,NaN,NaN,NaN,NaN
7,01.10.2020,Jim Knopf und die wilde 13,NaN,NaN,https://www.kino-zeit.de/node/48472,NaN,NaN,NaN,NaN
8,01.10.2020,Niemals Selten Manchmal Immer,NaN,NaN,https://www.kino-zeit.de/node/48347,NaN,NaN,NaN,NaN
9,02.10.2020,On the Rocks,NaN,NaN,https://www.kino-zeit.de/node/50572,NaN,NaN,NaN,NaN


In [ ]:
##NOTES
week_1=''.join([f'https://www.kino-zeit.de/filme/filmstarts/{i["href"]}' for i in soup.find_all("a",{"class": "next btn btn-primary"})])
soup= make_soup(week_1)
week_2=''.join([f'https://www.kino-zeit.de/filme/filmstarts/{i["href"]}' for i in soup.find_all("a",{"class": "next btn btn-primary"})])
soup= make_soup(week_2)

In [38]:
#AKTUELLE WOCHE WEEK 0
week_0='https://www.kino-zeit.de/filme/filmstarts/aktuell'
soup= make_soup(week_0)

In [39]:
#Filmtitel der jeweils aktuellen Seite
films_0_titel=[i.text for i in soup.find_all(['h3','a'],{'class': 'filmlink'})]
df_0=pd.DataFrame(films_0_titel)
df_0.columns = ['Filmtitel']
df_0

,Filmtitel
0,Contemporary Past - Die Gegenwart der Vergange...
1,Die Misswahl - Der Beginn einer Revolution (2020)
2,Drachenreiter (2020)
3,Enfant Terrible (2020)
4,Good Kisser (2019)
5,"Gott, du kannst ein Arsch sein! (2020)"
6,Hurtigruten und Norwegen (2020)
7,Jim Knopf und die wilde 13 (2020)
8,Niemals Selten Manchmal Immer (2020)
9,On the Rocks (2020)


In [40]:
#Link zur Filmseite
films_pglnk_0=[f'https://www.kino-zeit.de{i["href"]}' for i in soup.find_all(["h3","a"],{"class": "filmlink"})]
films_pglnk_0
df_0['page_link']=films_pglnk_0

In [41]:
#Starttermin
films_start_0=[i.text for i in soup.find_all('div',{'class':'col-md-2 infospalte-1'})]
films_start_0 = re.findall(r'\d\d.\d\d.\d\d\d\d', str(films_start_0))
df_0['Starttermin']=films_start_0
df_0

,Filmtitel,page_link,Starttermin
0,Contemporary Past - Die Gegenwart der Vergange...,https://www.kino-zeit.de/node/50404,01.10.2020
1,Die Misswahl - Der Beginn einer Revolution (2020),https://www.kino-zeit.de/node/48346,01.10.2020
2,Drachenreiter (2020),https://www.kino-zeit.de/node/49005,01.10.2020
3,Enfant Terrible (2020),https://www.kino-zeit.de/node/48025,01.10.2020
4,Good Kisser (2019),https://www.kino-zeit.de/node/50833,01.10.2020
5,"Gott, du kannst ein Arsch sein! (2020)",https://www.kino-zeit.de/node/50476,01.10.2020
6,Hurtigruten und Norwegen (2020),https://www.kino-zeit.de/node/50852,01.10.2020
7,Jim Knopf und die wilde 13 (2020),https://www.kino-zeit.de/node/48472,01.10.2020
8,Niemals Selten Manchmal Immer (2020),https://www.kino-zeit.de/node/48347,01.10.2020
9,On the Rocks (2020),https://www.kino-zeit.de/node/50572,02.10.2020


In [42]:
regie_list=[]
darsteller_list=[]
list_img_url=[]
list_verleih=[]
list_prodland=[]

for i in df_0['page_link']:   
    sub_page=make_soup(i)
    regie=[i.text.replace('\nRegie','').replace('\n\n','').split(',') for i in sub_page.find_all('div',{'class':'field field--name-field-regie field--type-entity-reference field--label-above'})]
    regie_list.append(regie[0])
    darsteller=[i.text.replace('\nDarsteller','').split('\n') for i in sub_page.find_all('div',{'class':'field field--name-field-hauptdarsteller field--type-entity-reference field--label-above'})]
    darsteller_list.append(darsteller)
    img_url=[i['data-big'] for i in sub_page.find_all('img',{'id':'headerbildbig'})]
    list_img_url.append(img_url)
    verleih=[i.text.replace('\nFilmverleih','').replace('\n\n','') for i in sub_page.find_all(['div','a'],{'class':'field field--name-field-distribution-film field--type-entity-reference field--label-above'})]
    list_verleih.append(verleih)
    prodland=[i.text.replace('\nProduktionsland','').replace('\n\n','') for i in sub_page.find_all(['div','a'],{'class':'field field--name-field-produktionsland field--type-entity-reference field--label-above'})]
    list_prodland.append(prodland)
    
df_0['Regie']=regie_list
df_0['Darsteller']=darsteller_list
df_0['Foto']=list_img_url
df_0['Verleih']=list_verleih
df_0['Produktionsland']=list_prodland

In [43]:
####WEEK_1#####

week_1=''.join([f'https://www.kino-zeit.de/filme/filmstarts/{i["href"]}' for i in soup.find_all("a",{"class": "next btn btn-primary"})])
soup= make_soup(week_1)

####FILMTITEL WEEK_1
films_1_titel=[i.text for i in soup.find_all(['h3','a'],{'class': 'filmlink'})]
df_1=pd.DataFrame(films_1_titel)
df_1.columns = ['Filmtitel']

# PAGELINKS WEEK 1
films_pglnk_1=[f'https://www.kino-zeit.de{i["href"]}' for i in soup.find_all(["h3","a"],{"class": "filmlink"})]
films_pglnk_1
df_1['page_link']=films_pglnk_1

#STARTTERMINE WEEK 1
films_start_1=[i.text for i in soup.find_all('div',{'class':'col-md-2 infospalte-1'})]
films_start_1 = re.findall(r'\d\d.\d\d.\d\d\d\d', str(films_start_1))
df_1['Starttermin']=films_start_1

#WEEK 1CHECKING REGIE, DARSTELLER, IMG, VERLEIH, PRODUNKTIONSLAND
regie_list=[]
darsteller_list=[]
list_img_url=[]
list_verleih=[]
list_prodland=[]

for i in df_1['page_link']:
    sub_page=make_soup(i)
    regie=[i.text.replace('\nRegie','').replace('\n\n','').split(',') for i in sub_page.find_all('div',{'class':'field field--name-field-regie field--type-entity-reference field--label-above'})]
    regie_list.append(regie[0])
    darsteller=[i.text.replace('\nDarsteller','').split('\n') for i in sub_page.find_all('div',{'class':'field field--name-field-hauptdarsteller field--type-entity-reference field--label-above'})]
    darsteller_list.append(darsteller)
    img_url=[i['data-big'] for i in sub_page.find_all('img',{'id':'headerbildbig'})]
    list_img_url.append(img_url)
    verleih=[i.text.replace('\nFilmverleih','').replace('\n\n','') for i in sub_page.find_all(['div','a'],{'class':'field field--name-field-distribution-film field--type-entity-reference field--label-above'})]
    list_verleih.append(verleih)
    prodland=[i.text.replace('\nProduktionsland','').replace('\n\n','') for i in sub_page.find_all(['div','a'],{'class':'field field--name-field-produktionsland field--type-entity-reference field--label-above'})]
    list_prodland.append(prodland)
    
df_1['Regie']=regie_list
df_1['Darsteller']=darsteller_list
df_1['Foto']=list_img_url
df_1['Verleih']=list_verleih
df_1['Produktionsland']=list_prodland

In [44]:
####WEEK 2 ####
##URL###
week_2=''.join([f'https://www.kino-zeit.de/filme/filmstarts/{i["href"]}' for i in soup.find_all("a",{"class": "next btn btn-primary"})])
soup= make_soup(week_2)

####FILMTITEL WEEK_2
films_2_titel=[i.text for i in soup.find_all(['h3','a'],{'class': 'filmlink'})]
df_2=pd.DataFrame(films_2_titel)
df_2.columns = ['Filmtitel']

# PAGELINKS WEEK 2
films_pglnk_2=[f'https://www.kino-zeit.de{i["href"]}' for i in soup.find_all(["h3","a"],{"class": "filmlink"})]
films_pglnk_2
df_2['page_link']=films_pglnk_2
df_2

#STARTTERMINE WEEK 2
films_start_2=[i.text for i in soup.find_all('div',{'class':'col-md-2 infospalte-1'})]
films_start_2 = re.findall(r'\d\d.\d\d.\d\d\d\d', str(films_start_2))
df_2['Starttermin']=films_start_2
df_2


#WEEK 2 CHECKING REGIE, DARSTELLER, IMG, VERLEIH, PRODUNKTIONSLAND
regie_list=[]
darsteller_list=[]
list_img_url=[]
list_verleih=[]
list_prodland=[]

for i in df_2['page_link']:
    sub_page=make_soup(i)
    regie=[i.text.replace('\nRegie','').replace('\n\n','').split(',') for i in sub_page.find_all('div',{'class':'field field--name-field-regie field--type-entity-reference field--label-above'})]
    regie_list.append(regie[0])
    darsteller=[i.text.replace('\nDarsteller','').split('\n') for i in sub_page.find_all('div',{'class':'field field--name-field-hauptdarsteller field--type-entity-reference field--label-above'})]
    darsteller_list.append(darsteller)
    img_url=[i['data-big'] for i in sub_page.find_all('img',{'id':'headerbildbig'})]
    list_img_url.append(img_url)
    verleih=[i.text.replace('\nFilmverleih','').replace('\n\n','') for i in sub_page.find_all(['div','a'],{'class':'field field--name-field-distribution-film field--type-entity-reference field--label-above'})]
    list_verleih.append(verleih)
    prodland=[i.text.replace('\nProduktionsland','').replace('\n\n','') for i in sub_page.find_all(['div','a'],{'class':'field field--name-field-produktionsland field--type-entity-reference field--label-above'})]
    list_prodland.append(prodland)
    
df_2['Regie']=regie_list
df_2['Darsteller']=darsteller_list
df_2['Foto']=list_img_url
df_2['Verleih']=list_verleih
df_2['Produktionsland']=list_prodland
df_2

,Filmtitel,page_link,Starttermin,Regie,Darsteller,Foto,Verleih,Produktionsland
0,Astronaut (2019),https://www.kino-zeit.de/node/50228,15.10.2020,[Shelagh McLeod],"[[, , Richard Dreyfuss, Lyriq Bent, Krista Bri...",[https://www.kino-zeit.de/sites/default/files/...,[JETS Filmverleih & Vertrieb],[Kanada]
1,Bruno - Die lange Heimreise (2019),https://www.kino-zeit.de/node/48854,15.10.2020,[Karl Golden],"[[, , Diarmaid Murtagh, Woody Norman, Seun Sho...",[https://www.kino-zeit.de/sites/default/files/...,[Filmperlen],[Großbritannien]
2,Chernobyl: Abyss (2020),https://www.kino-zeit.de/node/50781,18.10.2020,[Danila Kozlovsky],"[[, , Oksana Akinshina, Danila Kozlowskj, Fili...",[https://www.kino-zeit.de/sites/default/files/...,[Capelight Pictures / Central],[Russland]
3,Der Bär in mir (2019),https://www.kino-zeit.de/node/50292,15.10.2020,[Roman Droux],[],[https://www.kino-zeit.de/sites/default/files/...,[MFA + Filmdistribution],[Schweiz]
4,Der geheime Garten (2020),https://www.kino-zeit.de/node/47584,15.10.2020,[Marc Munden],"[[, , Colin Firth, Maeve Dermody, Julie Walter...",[https://www.kino-zeit.de/sites/default/files/...,[STUDIOCANAL GmbH Filmverleih],[Großbritannien\nFrankreich]
5,I Am Greta (2020),https://www.kino-zeit.de/node/50550,16.10.2020,[Nathan Grossman],"[[, , Greta Thunberg, , ], [, , Greta Thunberg...",[https://www.kino-zeit.de/sites/default/files/...,[Filmwelt Verleihagentur],[Schweden]
6,Making Montgomery Clift (2018),https://www.kino-zeit.de/node/47343,17.10.2020,[Robert Anderson Clift\nHillary Demmon],[],[https://www.kino-zeit.de/sites/default/files/...,[missingFILMs ],[USA]
7,Martin Margiela - Mythos der Mode (2019),https://www.kino-zeit.de/node/48816,15.10.2020,[Reiner Holzemer],[],[https://www.kino-zeit.de/sites/default/files/...,[Filmwelt Verleihagentur],[Deutschland\nBelgien]
8,Moskau Einfach! (2020),https://www.kino-zeit.de/node/48285,15.10.2020,[Micha Lewinsky],"[[, , Miriam Stein, Oriana Schrage, Urs Jucker...",[https://www.kino-zeit.de/sites/default/files/...,[Arsenal Filmverleih GmbH ],[Schweiz]
9,Mrs. Taylor's Singing Club (2019),https://www.kino-zeit.de/node/48466,15.10.2020,[Peter Cattaneo],"[[, , Kristin Scott Thomas, Sharon Horgan, Emm...",[https://www.kino-zeit.de/sites/default/files/...,[Leonine],[Großbritannien]


In [45]:
####WEEK 3 ####
##URL###
week_3=''.join([f'https://www.kino-zeit.de/filme/filmstarts/{i["href"]}' for i in soup.find_all("a",{"class": "next btn btn-primary"})])
soup= make_soup(week_3)

####FILMTITEL WEEK_3
films_3_titel=[i.text for i in soup.find_all(['h3','a'],{'class': 'filmlink'})]
df_3=pd.DataFrame(films_3_titel)
df_3.columns = ['Filmtitel']

# PAGELINKS WEEK 3
films_pglnk_3=[f'https://www.kino-zeit.de{i["href"]}' for i in soup.find_all(["h3","a"],{"class": "filmlink"})]
films_pglnk_3
df_3['page_link']=films_pglnk_3
df_3

#STARTTERMINE WEEK 3
films_start_3=[i.text for i in soup.find_all('div',{'class':'col-md-2 infospalte-1'})]
films_start_3 = re.findall(r'\d\d.\d\d.\d\d\d\d', str(films_start_3))
df_3['Starttermin']=films_start_3
df_3

#WEEK 3 CHECKING REGIE, DARSTELLER, IMG, VERLEIH, PRODUNKTIONSLAND
regie_list=[]
darsteller_list=[]
list_img_url=[]
list_verleih=[]
list_prodland=[]

for i in df_3['page_link']:
    sub_page=make_soup(i)
    regie=[i.text.replace('\nRegie','').replace('\n\n','').split(',') for i in sub_page.find_all('div',{'class':'field field--name-field-regie field--type-entity-reference field--label-above'})]
    regie_list.append(regie[0])
    darsteller=[i.text.replace('\nDarsteller','').split('\n') for i in sub_page.find_all('div',{'class':'field field--name-field-hauptdarsteller field--type-entity-reference field--label-above'})]
    darsteller_list.append(darsteller)
    img_url=[i['data-big'] for i in sub_page.find_all('img',{'id':'headerbildbig'})]
    list_img_url.append(img_url)
    verleih=[i.text.replace('\nFilmverleih','').replace('\n\n','') for i in sub_page.find_all(['div','a'],{'class':'field field--name-field-distribution-film field--type-entity-reference field--label-above'})]
    list_verleih.append(verleih)
    prodland=[i.text.replace('\nProduktionsland','').replace('\n\n','') for i in sub_page.find_all(['div','a'],{'class':'field field--name-field-produktionsland field--type-entity-reference field--label-above'})]
    list_prodland.append(prodland)
    
df_3['Regie']=regie_list
df_3['Darsteller']=darsteller_list
df_3['Foto']=list_img_url
df_3['Verleih']=list_verleih
df_3['Produktionsland']=list_prodland
df_3

,Filmtitel,page_link,Starttermin,Regie,Darsteller,Foto,Verleih,Produktionsland
0,Bohnenstange (2019),https://www.kino-zeit.de/node/46389,22.10.2020,[Kantemir Balagov],[],[https://www.kino-zeit.de/sites/default/files/...,[eksystent distribution filmverleih],[Russland]
1,Cortex (2020),https://www.kino-zeit.de/node/50644,22.10.2020,[Moritz Bleibtreu],"[[, , Moritz Bleibtreu, Jannis Niewöhner, Wota...",[https://www.kino-zeit.de/sites/default/files/...,[Warner Bros.],[Deutschland]
2,Die Stimme des Regenwaldes (2019),https://www.kino-zeit.de/node/47764,22.10.2020,[Niklaus Hilber],"[[, , Sven Schelker, Charlotte Heinimann, Matt...",[https://www.kino-zeit.de/sites/default/files/...,[Camino Filmverleih GmbH],[Schweiz\nÖsterreich]
3,Ema (2019),https://www.kino-zeit.de/node/44753,22.10.2020,[Pablo Larrain],"[[, , Gael Garcia Bernal, Mariana Di Girolamo,...",[https://www.kino-zeit.de/sites/default/files/...,[Koch Films GmbH\nStudiocanal],[Chile]
4,Kajillionaire (2020),https://www.kino-zeit.de/node/48596,22.10.2020,[Miranda July],"[[, , Evan Rachel Wood, Gina Rodriguez, Richar...",[https://www.kino-zeit.de/sites/default/files/...,[Universal Pictures International Germany GmbH],[USA]
5,Komm und sieh (1985),https://www.kino-zeit.de/node/50356,22.10.2020,[Elem Klimow],[],[https://www.kino-zeit.de/sites/default/files/...,[Drop-Out Cinema eG],[UdSSR]
6,"Mein Liebhaber, der Esel & ich (2020)",https://www.kino-zeit.de/node/50818,22.10.2020,[Caroline Vignal],"[[, , Laure Calamy, Benjamin Lavernhe, Olivia ...",[https://www.kino-zeit.de/sites/default/files/...,[Capelight Pictures / Wild Bunch],[Frankreich]
7,"Regeln am Band, bei hoher Geschwindigkeit (2020)",https://www.kino-zeit.de/node/48463,22.10.2020,[Yulia Lokshina],[],[https://www.kino-zeit.de/sites/default/files/...,[jip Film & Verleih],[Deutschland]
8,The Beach House (2019),https://www.kino-zeit.de/node/50175,22.10.2020,[Jeffrey A. Brown],"[[, , Liana Liberato, Noah Le Gros, Jake Weber...",[https://www.kino-zeit.de/sites/default/files/...,[Koch Films GmbH\n24 Bilder],[USA]
9,The Great Green Wall (2019),https://www.kino-zeit.de/node/48568,24.10.2020,[Jared P. Scott],[],[https://www.kino-zeit.de/sites/default/files/...,[Weltkino Filmverleih GmbH],[Großbritannien]


In [46]:
####WEEK 4 #####
##URL###
week_4=''.join([f'https://www.kino-zeit.de/filme/filmstarts/{i["href"]}' for i in soup.find_all("a",{"class": "next btn btn-primary"})])
soup= make_soup(week_4)

####FILMTITEL WEEK_3
films_4_titel=[i.text for i in soup.find_all(['h3','a'],{'class': 'filmlink'})]
df_4=pd.DataFrame(films_4_titel)
df_4.columns = ['Filmtitel']

# PAGELINKS WEEK 4
films_pglnk_4=[f'https://www.kino-zeit.de{i["href"]}' for i in soup.find_all(["h3","a"],{"class": "filmlink"})]
films_pglnk_4
df_4['page_link']=films_pglnk_4
df_4

#STARTTERMINE WEEK 4
films_start_4=[i.text for i in soup.find_all('div',{'class':'col-md-2 infospalte-1'})]
films_start_4 = re.findall(r'\d\d.\d\d.\d\d\d\d', str(films_start_4))
df_4['Starttermin']=films_start_4
df_4

#WEEK 4 CHECKING REGIE, DARSTELLER, IMG, VERLEIH, PRODUNKTIONSLAND
regie_list=[]
darsteller_list=[]
list_img_url=[]
list_verleih=[]
list_prodland=[]

for i in df_4['page_link']:
    sub_page=make_soup(i)
    regie=[i.text.replace('\nRegie','').replace('\n\n','').split(',') for i in sub_page.find_all('div',{'class':'field field--name-field-regie field--type-entity-reference field--label-above'})]
    regie_list.append(regie[0])
    darsteller=[i.text.replace('\nDarsteller','').split('\n') for i in sub_page.find_all('div',{'class':'field field--name-field-hauptdarsteller field--type-entity-reference field--label-above'})]
    darsteller_list.append(darsteller)
    img_url=[i['data-big'] for i in sub_page.find_all('img',{'id':'headerbildbig'})]
    list_img_url.append(img_url)
    verleih=[i.text.replace('\nFilmverleih','').replace('\n\n','') for i in sub_page.find_all(['div','a'],{'class':'field field--name-field-distribution-film field--type-entity-reference field--label-above'})]
    list_verleih.append(verleih)
    prodland=[i.text.replace('\nProduktionsland','').replace('\n\n','') for i in sub_page.find_all(['div','a'],{'class':'field field--name-field-produktionsland field--type-entity-reference field--label-above'})]
    list_prodland.append(prodland)
    
df_4['Regie']=regie_list
df_4['Darsteller']=darsteller_list
df_4['Foto']=list_img_url
df_4['Verleih']=list_verleih
df_4['Produktionsland']=list_prodland
df_4

,Filmtitel,page_link,Starttermin,Regie,Darsteller,Foto,Verleih,Produktionsland
0,Außer Atem (1960),https://www.kino-zeit.de/node/17390,29.10.2020,[Jean-Luc Godard],"[[, , Jean Seberg, Jean-Paul Belmondo, Henri-J...",[https://www.kino-zeit.de/sites/default/files/...,[Arthaus\nSTUDIOCANAL GmbH Filmverleih],[Frankreich]
1,Black Widow (2020),https://www.kino-zeit.de/node/48064,29.10.2020,[Cate Shortland],"[[, , Scarlett Johansson, Florence Pugh, David...",[https://www.kino-zeit.de/sites/default/files/...,[The Walt Disney Company Germany GmbH],[USA]
2,Clara und der magische Drache (2019),https://www.kino-zeit.de/node/50059,29.10.2020,[Aleksandr Klymenko],[],[https://www.kino-zeit.de/sites/default/files/...,[Lakeside Film],[Ukraine]
3,Der Boandlkramer und die ewige Liebe (2020),https://www.kino-zeit.de/node/48036,29.10.2020,[Joseph Vilsmaier],"[[, , Michael Bully Herbig, Hape Kerkeling, Ha...",[https://www.kino-zeit.de/sites/default/files/...,[Leonine],[Deutschland]
4,Glitzer & Staub (2020),https://www.kino-zeit.de/node/48512,29.10.2020,[Anna Koch\nJulia Lemke],[],[https://www.kino-zeit.de/sites/default/files/...,[Port-Au-Prince Pictures\n24 Bilder],[Deutschland]
5,"Ruben Brandt, Collector (2018)",https://www.kino-zeit.de/node/42959,29.10.2020,[Milorad Krstic],"[[, , Gabriella Hámori, Ivan Kamarás, Zalán Ma...",[https://www.kino-zeit.de/sites/default/files/...,[Drop-out Cinema],[Ungarn]
6,Schlaf (2020),https://www.kino-zeit.de/node/48752,29.10.2020,[Michael Venus],"[[, , Sandra Hüller, Gro Swantje Kohlhof, Mari...",[https://www.kino-zeit.de/sites/default/files/...,[Salzgeber],[Deutschland]
7,Schwesterlein (2020),https://www.kino-zeit.de/node/48704,29.10.2020,[Stéphanie Chuat\nVéronique Reymond],"[[, , Nina Hoss, Lars Eidinger, Marthe Keller,...",[https://www.kino-zeit.de/sites/default/files/...,[Weltkino Filmverleih],[Schweiz]
8,Srbenka (2018),https://www.kino-zeit.de/node/43411,29.10.2020,[Nebojša Slijepčević],[],[https://www.kino-zeit.de/sites/default/files/...,[Rise and Shine Cinema],[Kroatien]
9,The Booksellers - Aus Liebe zum Buch (2019),https://www.kino-zeit.de/node/47837,29.10.2020,[D.W. Young],[],[https://www.kino-zeit.de/sites/default/files/...,[Mindjazz Pictures],[USA]


In [121]:
df_filmstarts=pd.concat([df_0, df_1, df_2, df_3, df_4], ignore_index=True)
df_filmstarts.to_csv('Filmstarts.csv')
df_filmstarts

,Filmtitel,page_link,Starttermin,Regie,Darsteller,Foto,Verleih,Produktionsland
0,Contemporary Past - Die Gegenwart der Vergange...,https://www.kino-zeit.de/node/50404,01.10.2020,[Kamil Majchrzak],[],[https://www.kino-zeit.de/sites/default/files/...,[Les Funambules Filmproduktion],[Polen]
1,Die Misswahl - Der Beginn einer Revolution (2020),https://www.kino-zeit.de/node/48346,01.10.2020,[Philippa Lowthorpe],"[[, , Keira Knightley, Gugu Mbatha-Raw, Jessie...",[https://www.kino-zeit.de/sites/default/files/...,[eOne \nParamount Pictures Germany GmbH],[Großbritannien]
2,Drachenreiter (2020),https://www.kino-zeit.de/node/49005,01.10.2020,[Tomer Eshed],[],[https://www.kino-zeit.de/sites/default/files/...,[Constantin Filmverleih],[Deutschland]
3,Enfant Terrible (2020),https://www.kino-zeit.de/node/48025,01.10.2020,[Oskar Roehler],"[[, , Oliver Masucci, Katja Riemann, Hary Prin...",[https://www.kino-zeit.de/sites/default/files/...,[Weltkino Filmverleih],[]
4,Good Kisser (2019),https://www.kino-zeit.de/node/50833,01.10.2020,[Wendy Jo Carlton],"[[, , Kari Alison Hodge, Julia Eringer, Courtn...",[https://www.kino-zeit.de/sites/default/files/...,[Pro-Fun Media Filmverleih],[USA]
...,...,...,...,...,...,...,...,...
57,Srbenka (2018),https://www.kino-zeit.de/node/43411,29.10.2020,[Nebojša Slijepčević],[],[https://www.kino-zeit.de/sites/default/files/...,[Rise and Shine Cinema],[Kroatien]
58,The Booksellers - Aus Liebe zum Buch (2019),https://www.kino-zeit.de/node/47837,29.10.2020,[D.W. Young],[],[https://www.kino-zeit.de/sites/default/files/...,[Mindjazz Pictures],[USA]
59,Und morgen die ganze Welt (2020),https://www.kino-zeit.de/node/50396,29.10.2020,[Julia von Heinz],"[[, , Mala Emde, Tonio Schneider, Luisa-Céline...",[https://www.kino-zeit.de/sites/default/files/...,[Alamode Filmverleih],[Deutschland]
60,Wildherz - Auf der Reise zu mir selbst (2020),https://www.kino-zeit.de/node/50377,29.10.2020,[Caro Lobig],[],[https://www.kino-zeit.de/sites/default/files/...,[catamaranfilms\nbarnsteiner-film ],[Deutschland]
